https://www.bgc-jena.mpg.de/wetter/

https://machinelearningmastery.com/the-attention-mechanism-from-scratch/

https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/

https://towardsdatascience.com/time-series-forecasting-with-deep-learning-and-attention-mechanism-2d001fc871fc

# Skip Connection & Attention Mechanism Implementation

In [1]:
# import required libraries

# data preprocessing
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import IsolationForest
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# visualization
from matplotlib import pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = (15.0, 6.0)

# implementing deep learning model
import tensorflow as ts
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout,LSTM,Input,Layer, Concatenate, Activation, Multiply, Add, Lambda, Permute, Reshape
from tensorflow.keras.optimizers import Adam

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model 

## Data Preprocessing

In [2]:
data = pd.read_csv("mpi_roof\mpi_roof.csv",encoding= 'unicode_escape')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31931 entries, 0 to 31930
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date Time             31931 non-null  object 
 1   p (mbar)              31931 non-null  float64
 2   T (degC)              31931 non-null  float64
 3   Tpot (K)              31931 non-null  float64
 4   Tdew (degC)           31931 non-null  float64
 5   rh (%)                31931 non-null  float64
 6   VPmax (mbar)          31931 non-null  float64
 7   VPact (mbar)          31931 non-null  float64
 8   VPdef (mbar)          31931 non-null  float64
 9   sh (g/kg)             31931 non-null  float64
 10  H2OC (mmol/mol)       31931 non-null  float64
 11  rho (g/m**3)          31931 non-null  float64
 12  wv (m/s)              31931 non-null  float64
 13  max. wv (m/s)         31931 non-null  float64
 14  wd (deg)              31931 non-null  float64
 15  rain (mm)          

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm)
count,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,...,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000,31931.000000
mean,990.646484,10.966426,284.885015,6.840672,79.511083,15.146275,10.647325,4.498847,6.718122,10.750479,...,2.057666,3.385259,172.644516,0.009580,31.957032,106.328058,208.339513,250.871895,19.939707,424.905399
std,8.268963,8.751587,8.807894,6.027105,19.419032,8.942102,4.077635,6.807391,2.587192,4.123436,...,1.448510,2.203478,83.729286,0.167079,123.946813,193.646628,376.554554,479.030428,8.154881,303.832296
min,955.430000,-13.390000,259.340000,-14.190000,14.230000,2.170000,2.030000,0.000000,1.260000,2.020000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9999.000000,6.410000,-9999.000000
25%,985.930000,4.610000,278.640000,2.890000,68.700000,8.490000,7.520000,0.290000,4.730000,7.590000,...,0.990000,1.750000,119.500000,0.000000,0.000000,0.000000,0.000000,0.000000,13.630000,420.100000
50%,991.160000,11.070000,285.160000,7.330000,84.200000,13.200000,10.250000,1.820000,6.470000,10.360000,...,1.710000,2.850000,195.100000,0.000000,0.000000,0.000000,0.000000,0.000000,19.150000,430.100000
75%,995.540000,17.025000,290.870000,11.465000,96.200000,19.435000,13.560000,5.580000,8.550000,13.680000,...,2.740000,4.490000,224.950000,0.000000,0.000000,121.430000,241.275000,288.995000,25.195000,444.300000
max,1014.630000,37.950000,312.210000,19.720000,100.000000,66.130000,23.010000,56.720000,14.670000,23.380000,...,12.810000,17.820000,360.000000,19.200000,600.000000,1044.180000,2026.210000,2485.400000,45.800000,530.200000


In [3]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True,drop=True)
data.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm)
0,01.07.2022 00:10:00,984.33,22.42,296.91,19.72,84.7,27.17,23.01,4.16,14.67,...,1.18,1.63,219.9,0.0,0,0.0,0.0,0.0,31.74,447.2
1,01.07.2022 00:20:00,984.36,22.02,296.50,19.72,86.8,26.51,23.01,3.50,14.67,...,1.02,1.58,222.4,0.0,0,0.0,0.0,0.0,31.63,445.6
2,01.07.2022 00:30:00,984.35,21.77,296.25,19.68,87.9,26.11,22.95,3.16,14.63,...,0.99,1.64,191.0,0.0,0,0.0,0.0,0.0,31.52,443.1
3,01.07.2022 00:40:00,984.27,22.00,296.49,19.47,85.6,26.48,22.67,3.81,14.45,...,1.45,2.40,211.8,0.0,0,0.0,0.0,0.0,31.44,443.4
4,01.07.2022 00:50:00,984.36,22.23,296.71,19.40,84.0,26.86,22.56,4.30,14.38,...,1.66,2.59,211.9,0.0,0,0.0,0.0,0.0,31.37,444.4


In [4]:
data["Date"] = pd.to_datetime(data["Date Time"])

# data["Date"] = data["Date Time"].astype("datetime64[ns]")
data["Date"].shape

(31930,)

In [5]:
data.set_index("Date Time",inplace=True)
data.drop("Date",axis=1,inplace=True)
data.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm)
Date Time,,,,,,,,,,,,,,,,,,,,,
01.07.2022 00:10:00,984.33,22.42,296.91,19.72,84.7,27.17,23.01,4.16,14.67,23.38,...,1.18,1.63,219.9,0.0,0,0.0,0.0,0.0,31.74,447.2
01.07.2022 00:20:00,984.36,22.02,296.50,19.72,86.8,26.51,23.01,3.50,14.67,23.38,...,1.02,1.58,222.4,0.0,0,0.0,0.0,0.0,31.63,445.6
01.07.2022 00:30:00,984.35,21.77,296.25,19.68,87.9,26.11,22.95,3.16,14.63,23.32,...,0.99,1.64,191.0,0.0,0,0.0,0.0,0.0,31.52,443.1
01.07.2022 00:40:00,984.27,22.00,296.49,19.47,85.6,26.48,22.67,3.81,14.45,23.03,...,1.45,2.40,211.8,0.0,0,0.0,0.0,0.0,31.44,443.4
01.07.2022 00:50:00,984.36,22.23,296.71,19.40,84.0,26.86,22.56,4.30,14.38,22.92,...,1.66,2.59,211.9,0.0,0,0.0,0.0,0.0,31.37,444.4


In [6]:
# feature selection
data_selected = data.iloc[:, [0,1,4,8,16,20]]
display(data_selected.head())

,p (mbar),T (degC),rh (%),sh (g/kg),SWDR (W/m²),CO2 (ppm)
Date Time,,,,,,
01.07.2022 00:10:00,984.33,22.42,84.7,14.67,0.0,447.2
01.07.2022 00:20:00,984.36,22.02,86.8,14.67,0.0,445.6
01.07.2022 00:30:00,984.35,21.77,87.9,14.63,0.0,443.1
01.07.2022 00:40:00,984.27,22.00,85.6,14.45,0.0,443.4
01.07.2022 00:50:00,984.36,22.23,84.0,14.38,0.0,444.4


In [7]:
col_names = list(data_selected.columns)
colors = ['b','g','r','c','m','y']

In [8]:
# for i in range(data_selected.shape[1]):
#     plt.figure(figsize=(15,6))
#     plt.plot(data_selected.iloc[:,i],color=colors[i])
#     plt.title(col_names[i])
#     plt.show()

In [9]:
# scale data
scaler = MinMaxScaler(feature_range=(-1,1))
data_scaled = scaler.fit_transform(data_selected)


In [10]:
# train and test split
test_size = 1008
train = data_scaled[:-test_size,:]
test = data_scaled[-test_size:,:]


In [11]:
print(train.size)
print(test.size)

185532
6048


## Amonaly Handling

In [12]:
def plot_histogram(y,x,train,column_names,colors):
    plt.figure(figsize=(20,20))
    for i in range(train.shape[1]):
        plt.subplot(y,x,i+1)
        plt.hist(train[:,i],color=colors[i])
        plt.title(col_names[i])

    plt.show()

In [13]:
def plot_boxplot(y,x,train,column_names,colors):
    plt.figure(figsize=(20,20))
    for i in range(train.shape[1]):
        plt.subplot(y,x,i+1)
        plt.boxplot(train[:,i])
        plt.title(col_names[i])

    plt.show()

In [14]:
# plot_histogram(3,2,train,col_names,colors)

In [15]:
# plot_boxplot(3,2,train,col_names,colors)

In [16]:
def remove_anomaly(train,index,anomaly_idx_list):
    for i in range(len(anomaly_idx_list)):
        size = 0
        mean = 0
        for j in range(anomaly_idx_list[i]-100,anomaly_idx_list[i]+101):
            if(train.shape[0]-101>=j>=0 and j not in anomaly_idx_list ):
                size += 1
                mean += train[j,index]
#         print(mean/size)
        if size != 0:
            train[anomaly_idx_list[i],index] = mean/size
    return train

In [17]:
isolation_forest = IsolationForest(contamination=0.01)

for idx in range(train.shape[1]):
    dt_feature = [[item] for item in train[:,idx]]
    isolation_forest.fit(dt_feature)
    pred = isolation_forest.predict(dt_feature)
    anomaly = [i for i in range(len(pred)) if pred[i] == -1]
    
    train = remove_anomaly(train,idx,anomaly)
    
    anomaly_val = []
    for i in range(len(anomaly)):
        anomaly_val.append(dt_feature[anomaly[i]])
#     plt.plot(dt_feature,color='b')
#     plt.title(col_names[idx])
#     plt.scatter(anomaly,anomaly_val,color='r')
#     plt.legend(['original','anomaly'], loc='upper right')
#     plt.show()



In [18]:
# print(train)
# plot_boxplot(3,2,train,col_names,colors)

In [19]:
# for i in range(train.shape[1]):
#     plt.plot(train[:,i],color=colors[i])
#     plt.title(col_names[i])
#     plt.show()


In [20]:
# for i in range(test.shape[1]):
#     plt.plot(test[:,i],color=colors[i])
#     plt.title(col_names[i])
#     plt.show()

## LSTM Model Implementation

In [21]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(0,len(sequences),n_steps_out):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [22]:
n_features = 6
n_steps_in = 150
n_steps_out = 100
X_train, y_train = split_sequences(train,n_steps_in,n_steps_out)
X_test, y_test = split_sequences(test,n_steps_in,n_steps_out)

In [23]:
print(train.shape)
print(test.shape)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(30922, 6)
(1008, 6)
(307, 150, 6) (307, 100, 6)
(8, 150, 6) (8, 100, 6)


In [24]:
# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch % 150 == 0 and epoch > 0:
        lr = lr * 0.9   
    return lr

# early stopping and model chackpoint
learning_rate_scheduler = LearningRateScheduler(lr_scheduler)
early_stopping = EarlyStopping(monitor="val_loss",verbose=1,patience=50, mode="min",min_delta=0.0001)
model_checkpoint = ModelCheckpoint("best_model.h5",monitor="val_loss",verbose=1,save_best_only=True)


In [25]:
from keras import backend as K
inputs = Input(shape=(n_steps_in,n_features))

# Define LSTM layers with skip connections
lstm1 = LSTM(128, return_sequences=True)(inputs)
lstm2 = LSTM(64)(Concatenate()([inputs, lstm1]))

# define self-attention layer
attention = Dense(1, activation='tanh')(lstm1)
attention = Dropout(0.2)(attention)
attention = Dense(1, activation='softmax')(attention)

# Apply attention weights to LSTM outputs
context_vector = Multiply()([attention, lstm1])
context_vector = Lambda(lambda x: K.sum(x, axis=1))(context_vector)

# Concatenate context vector with output of second LSTM layer
concatenated = Concatenate()([context_vector, lstm2])

# Define output layer for 100 future timesteps with 6 features
outputs = Dense(100*6)(concatenated)
outputs = Reshape((100, 6))(outputs)

# Define model with skip connections and self-attention mechanism
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# 
model.compile(optimizer=Adam(learning_rate=0.00001), loss="mse")
epochs = 10000
validation_split = 0.3 # 30% validation set
history = model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=147 ,verbose=1,validation_split=validation_split,shuffle=False, callbacks=[learning_rate_scheduler,early_stopping,model_checkpoint])

Epoch 1/10000
2/2 [==============================] - ETA: 0s - loss: 62.5484
Epoch 1: val_loss improved from inf to 106.87645, saving model to best_model.h5
2/2 [==============================] - 11s 3s/step - loss: 62.5484 - val_loss: 106.8764 - lr: 1.0000e-05
Epoch 2/10000
2/2 [==============================] - ETA: 0s - loss: 61.8904
Epoch 2: val_loss improved from 106.87645 to 105.79816, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 61.8904 - val_loss: 105.7982 - lr: 1.0000e-05
Epoch 3/10000
2/2 [==============================] - ETA: 0s - loss: 61.2539
Epoch 3: val_loss improved from 105.79816 to 104.72611, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 61.2539 - val_loss: 104.7261 - lr: 1.0000e-05
Epoch 4/10000
2/2 [==============================] - ETA: 0s - loss: 60.6262
Epoch 4: val_loss improved from 104.72611 to 103.66530, saving model to best_model.h5
2/2 [==============================] - 3s

Epoch 32/10000
2/2 [==============================] - ETA: 0s - loss: 46.4714
Epoch 32: val_loss improved from 80.32088 to 79.62902, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 46.4714 - val_loss: 79.6290 - lr: 1.0000e-05
Epoch 33/10000
2/2 [==============================] - ETA: 0s - loss: 46.0701
Epoch 33: val_loss improved from 79.62902 to 78.94708, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 46.0701 - val_loss: 78.9471 - lr: 1.0000e-05
Epoch 34/10000
2/2 [==============================] - ETA: 0s - loss: 45.6745
Epoch 34: val_loss improved from 78.94708 to 78.27480, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 45.6745 - val_loss: 78.2748 - lr: 1.0000e-05
Epoch 35/10000
2/2 [==============================] - ETA: 0s - loss: 45.2844
Epoch 35: val_loss improved from 78.27480 to 77.61195, saving model to best_model.h5
2/2 [==============================] - 

Epoch 63/10000
2/2 [==============================] - ETA: 0s - loss: 36.1830
Epoch 63: val_loss improved from 62.62260 to 62.15557, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 36.1830 - val_loss: 62.1556 - lr: 1.0000e-05
Epoch 64/10000
2/2 [==============================] - ETA: 0s - loss: 35.9108
Epoch 64: val_loss improved from 62.15557 to 61.69371, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 35.9108 - val_loss: 61.6937 - lr: 1.0000e-05
Epoch 65/10000
2/2 [==============================] - ETA: 0s - loss: 35.6416
Epoch 65: val_loss improved from 61.69371 to 61.23695, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 35.6416 - val_loss: 61.2369 - lr: 1.0000e-05
Epoch 66/10000
2/2 [==============================] - ETA: 0s - loss: 35.3753
Epoch 66: val_loss improved from 61.23695 to 60.78518, saving model to best_model.h5
2/2 [==============================] - 

Epoch 94/10000
2/2 [==============================] - ETA: 0s - loss: 28.9310
Epoch 94: val_loss improved from 50.20200 to 49.86129, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 28.9310 - val_loss: 49.8613 - lr: 1.0000e-05
Epoch 95/10000
2/2 [==============================] - ETA: 0s - loss: 28.7317
Epoch 95: val_loss improved from 49.86129 to 49.52372, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 28.7317 - val_loss: 49.5237 - lr: 1.0000e-05
Epoch 96/10000
2/2 [==============================] - ETA: 0s - loss: 28.5342
Epoch 96: val_loss improved from 49.52372 to 49.18926, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 28.5342 - val_loss: 49.1893 - lr: 1.0000e-05
Epoch 97/10000
2/2 [==============================] - ETA: 0s - loss: 28.3385
Epoch 97: val_loss improved from 49.18926 to 48.85786, saving model to best_model.h5
2/2 [==============================] - 

2/2 [==============================] - 3s 2s/step - loss: 23.6531 - val_loss: 40.9274 - lr: 1.0000e-05
Epoch 125/10000
2/2 [==============================] - ETA: 0s - loss: 23.4992
Epoch 125: val_loss improved from 40.92735 to 40.66686, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 23.4992 - val_loss: 40.6669 - lr: 1.0000e-05
Epoch 126/10000
2/2 [==============================] - ETA: 0s - loss: 23.3465
Epoch 126: val_loss improved from 40.66686 to 40.40846, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 23.3465 - val_loss: 40.4085 - lr: 1.0000e-05
Epoch 127/10000
2/2 [==============================] - ETA: 0s - loss: 23.1950
Epoch 127: val_loss improved from 40.40846 to 40.15214, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 23.1950 - val_loss: 40.1521 - lr: 1.0000e-05
Epoch 128/10000
2/2 [==============================] - ETA: 0s - loss: 23.0447
Epoch 128: val

...

2/2 [==============================] - ETA: 0s - loss: 2.6081
Epoch 650: val_loss improved from 4.69954 to 4.68838, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.6081 - val_loss: 4.6884 - lr: 6.5610e-06
Epoch 651/10000
2/2 [==============================] - ETA: 0s - loss: 2.6022
Epoch 651: val_loss improved from 4.68838 to 4.67725, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.6022 - val_loss: 4.6773 - lr: 6.5610e-06
Epoch 652/10000
2/2 [==============================] - ETA: 0s - loss: 2.5964
Epoch 652: val_loss improved from 4.67725 to 4.66616, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.5964 - val_loss: 4.6662 - lr: 6.5610e-06
Epoch 653/10000
2/2 [==============================] - ETA: 0s - loss: 2.5906
Epoch 653: val_loss improved from 4.66616 to 4.65512, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.5906 

2/2 [==============================] - ETA: 0s - loss: 2.4367
Epoch 681: val_loss improved from 4.37132 to 4.36132, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.4367 - val_loss: 4.3613 - lr: 6.5610e-06
Epoch 682/10000
2/2 [==============================] - ETA: 0s - loss: 2.4315
Epoch 682: val_loss improved from 4.36132 to 4.35136, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.4315 - val_loss: 4.3514 - lr: 6.5610e-06
Epoch 683/10000
2/2 [==============================] - ETA: 0s - loss: 2.4263
Epoch 683: val_loss improved from 4.35136 to 4.34143, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.4263 - val_loss: 4.3414 - lr: 6.5610e-06
Epoch 684/10000
2/2 [==============================] - ETA: 0s - loss: 2.4211
Epoch 684: val_loss improved from 4.34143 to 4.33154, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.4211 

2/2 [==============================] - ETA: 0s - loss: 2.2837
Epoch 712: val_loss improved from 4.07712 to 4.06814, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.2837 - val_loss: 4.0681 - lr: 6.5610e-06
Epoch 713/10000
2/2 [==============================] - ETA: 0s - loss: 2.2791
Epoch 713: val_loss improved from 4.06814 to 4.05920, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.2791 - val_loss: 4.0592 - lr: 6.5610e-06
Epoch 714/10000
2/2 [==============================] - ETA: 0s - loss: 2.2744
Epoch 714: val_loss improved from 4.05920 to 4.05029, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.2744 - val_loss: 4.0503 - lr: 6.5610e-06
Epoch 715/10000
2/2 [==============================] - ETA: 0s - loss: 2.2698
Epoch 715: val_loss improved from 4.05029 to 4.04141, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.2698 

2/2 [==============================] - ETA: 0s - loss: 2.1467
Epoch 743: val_loss improved from 3.81272 to 3.80464, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.1467 - val_loss: 3.8046 - lr: 6.5610e-06
Epoch 744/10000
2/2 [==============================] - ETA: 0s - loss: 2.1425
Epoch 744: val_loss improved from 3.80464 to 3.79659, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 2.1425 - val_loss: 3.7966 - lr: 6.5610e-06
Epoch 745/10000
2/2 [==============================] - ETA: 0s - loss: 2.1383
Epoch 745: val_loss improved from 3.79659 to 3.78857, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.1383 - val_loss: 3.7886 - lr: 6.5610e-06
Epoch 746/10000
2/2 [==============================] - ETA: 0s - loss: 2.1342
Epoch 746: val_loss improved from 3.78857 to 3.78058, saving model to best_model.h5
2/2 [==============================] - 3s 1s/step - loss: 2.1342 

2/2 [==============================] - ETA: 0s - loss: 2.0321
Epoch 774: val_loss improved from 3.59125 to 3.58463, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 2.0321 - val_loss: 3.5846 - lr: 5.9049e-06
Epoch 775/10000
2/2 [==============================] - ETA: 0s - loss: 2.0287
Epoch 775: val_loss improved from 3.58463 to 3.57803, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 2.0287 - val_loss: 3.5780 - lr: 5.9049e-06
Epoch 776/10000
2/2 [==============================] - ETA: 0s - loss: 2.0252
Epoch 776: val_loss improved from 3.57803 to 3.57145, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 2.0252 - val_loss: 3.5715 - lr: 5.9049e-06
Epoch 777/10000
2/2 [==============================] - ETA: 0s - loss: 2.0218
Epoch 777: val_loss improved from 3.57145 to 3.56489, saving model to best_model.h5
2/2 [==============================] - 3s 2s/step - loss: 2.0218 

In [ ]:
# plot the validation and training loss
plt.figure(figsize=(15,6))
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Train vs Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Prediction

In [ ]:
# load model
model = load_model('best_model.h5')

In [ ]:
y_train = y_train.reshape(y_train.shape[0],n_steps_out*n_features)
y_test = y_test.reshape(y_test.shape[0],n_steps_out*n_features)
print(y_train.shape)
print(y_test.shape)

In [ ]:
"""
    1. Create y_pred array and X_test_model.
    2. y_pred is for saving predicted test values. Its shape is the same as y_test.
    3. X_test_model is for step by step prediction. Firstly X_test_model is equal to 
       the first input of LSTM model which is X_test[0]. X_test[0]'s shape is equal to (n_steps_in,n_features).
"""

y_pred = np.zeros((y_test.shape[0],y_test.shape[1]))
X_test_model = X_test[0].copy()


In [ ]:
"""
    Predict y values of test set step by step. At the beginning X_test_model is equal to X_test[0].
    Predicted values are store in "pred" variable. Then predicted values are copied to "y_pred" variable's 
    corresponding row. After that X_test_model is shifted by n_steps_out and the last n_steps_out number of values
    in X_test_model replaced with predicted values.
    
"""
for i in range(X_test.shape[0]):

    X_test_model = X_test_model.reshape(1,n_steps_in,n_features)
    pred = model.predict(X_test_model, verbose=0)
    X_test_model.reshape(n_steps_in,n_features)
    y_pred[i]=pred.reshape(1,n_steps_out*n_features )
    pred=pred.reshape(n_steps_out,n_features)
    if(n_steps_out>n_steps_in):
        X_test_model[0,:] = pred[-n_steps_in:]
    else:
        X_test_model[0,:-n_steps_out] = X_test_model[0,n_steps_out:]
        X_test_model[0,-n_steps_out:] = pred
    

In [ ]:
# inverse scaling
y_pred_inv = np.zeros_like(y_pred)
y_test_inv = np.zeros_like(y_test)
for i in range(1,n_steps_out+1):
    y_pred_inv[:,(i-1)*n_features:i*n_features] = scaler.inverse_transform(y_pred[:,(i-1)*n_features:i*n_features])
    
    y_test_inv[:,(i-1)*n_features:i*n_features] = scaler.inverse_transform(y_test[:,(i-1)*n_features:i*n_features])

    rmse = mean_squared_error(y_test_inv, y_pred_inv) ** (0.5)
print('Test RMSE: %.3f ' % rmse )

In [ ]:
# reshape y sets and calculate mse
y_test_re = y_test_inv.reshape(n_steps_out*y_test_inv.shape[0], n_features)
y_pred_re = y_pred_inv.reshape(n_steps_out*y_pred_inv.shape[0], n_features)
print(y_pred_re.shape)
print(y_test_re.shape)
rmse = mean_squared_error(y_test_re, y_pred_re) ** (0.5)
print('Test RMSE: %.3f ' % rmse )

In [ ]:
# print(y_pred_re[:10,])
# print(y_test_re[:10,])

In [ ]:
# plot predicted and actual y values of the test set
for i in range(n_features):
    plt.plot(y_test_re[:,i],color='black')
    plt.plot(y_pred_re[:,i],color='red')
    plt.legend(["test","predicted"])
    plt.title(col_names[i])
    plt.show()
